# **US ELECTION ANALYSIS WITH SQL**

## 1.0 Business Understanding
The task is to help a candidate to become US president. The winner of the election will be the candidate winning the most grand electors. 
Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win (roughly, but not exactly, proportional to the size of the state) and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.
Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. We assume here there are only 2 candidates, and no history (no trend of certain states to vote for a particular candidate or party). Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.

## 2.0 Data Understanding
At this moment, the team has no plans to purchase external databases or spend money
conducting surveys. This is because brute force computation of all possibilities to find the absolute optimum would be too computationally expensive.

Below are two datasets that we shall be considered for research:
* Grand electors by State. [Click Here](https://drive.google.com/a/moringaschool.com/file/d/1AuPlNXKjmEdRTLGqKbP-OP1XZ5UQPIPw/view?usp=sharing)
* Population by State. [Click Here](https://drive.google.com/a/moringaschool.com/file/d/1VKt_hF2pRqPxcNb1DKotkVXWNd2HX_KL/view?usp=sharing)

### 2.1 Exploring data
The Population datasets we shall be using contains two columns, State and Population respectively with a total record of 57 states/ territories in the United State of America. The Grand Electors by state dataset contains two columns, State and GrandElectors with a record of 52 states/ territories in the United State of America.



### 2.2 Verifying data quality
We have noted from the datasets that states are not capitalized the same way in both tables (one is in uppercase letters, the other not), so you will need to convert all to uppercase, for instance’
We also have no missing records on the data.

## 3.0 Data Preparation
* We shall be joining the two tables using the State field as the key and making a new table by the name “Election”. 
* The team wants to change the name of the "District of Columbia" state to its short version "DC", we shall be updating the records accordingly.
* We shall create a new column by the name “Ratio” to  compute the ratio between the number of grand electors and the population.
* We shall order the states by decreasing ratio of Grand Electors per capita. This will make our priority list.
* We shall compute the running total of Grand Electors in that sorted list.
* We shall also compute the half of the total of Grand Electors overall (in the whole country). This will enable us to determine the threshold we need to reach for winning the presidential election.
* We shall filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states will be ignored).

## **Connecting to our Database**

In [1]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

'Connected: @None'

In [2]:
# Importing Pandas

import pandas as pd

## **Loading our Datasets**

In [6]:
# Loading our GrandElectors_by_state dataset below

with open('GrandElectors_by_state.csv','r') as f:
    GrandElectors = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS GrandElectors;
%sql PERSIST GrandElectors;
%sql SELECT * FROM GrandElectors LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [7]:
# Loading our Population_by_state dataset below

with open('Population_by_state.csv','r') as f:
    Population = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS Population;
%sql PERSIST Population;


%sql SELECT * FROM Population LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


### Cleaning the dataset

In [9]:
# Converting State to uppercase

In [10]:
# GrandElector Dataset

%%sql 
UPDATE GrandElectors
SET State = upper(State);

SELECT State FROM GrandElectors LIMIT 5;

 * sqlite://
51 rows affected.
Done.


State
ALABAMA
ALASKA
ARIZONA
ARKANSAS
CALIFORNIA


In [11]:
# Population Dataset

%%sql 
UPDATE Population
SET State = upper(State);

SELECT State FROM Population LIMIT 5;

 * sqlite://
56 rows affected.
Done.


State
ALABAMA
ALASKA
AMERICAN SAMOA
ARIZONA
ARKANSAS


In [12]:
# Joining the GrandElectors and Population datasets

%%sql
CREATE TABLE Election AS SELECT State, Population, GrandElectors 
FROM (SELECT * FROM Population INNER JOIN GrandElectors ON  Population.State = GrandElectors.State);

SELECT * FROM Election LIMIT 5;

 * sqlite://
Done.
Done.


State,Population,GrandElectors
CALIFORNIA,39144818,55
TEXAS,27469114,38
FLORIDA,20271272,29
NEW YORK,19795791,29
ILLINOIS,12859995,20


In [13]:
# Changing the name of the "District of Columbia" state to its short version "DC"

%%sql

UPDATE Election
SET State = 'DC'
WHERE State = 'DISTRICT OF COLUMBIA';

SELECT * FROM Election;

 * sqlite://
1 rows affected.
Done.


State,Population,GrandElectors
CALIFORNIA,39144818,55
TEXAS,27469114,38
FLORIDA,20271272,29
NEW YORK,19795791,29
ILLINOIS,12859995,20
PENNSYLVANIA,12802503,20
OHIO,11613423,18
GEORGIA,10214860,16
NORTH CAROLINA,10042802,15
MICHIGAN,9922576,16


In [14]:
# We are computing the ratio between the number of grand electors and the population. 

# We shall create a new column with that ratio.

%%sql

ALTER TABLE Election 
ADD Ratio float(10,2)

 * sqlite://
Done.


[]

In [15]:
# Checking the status of the new column
%%sql 
SELECT * FROM Election LIMIT 5;

 * sqlite://
Done.


State,Population,GrandElectors,Ratio
CALIFORNIA,39144818,55,None
TEXAS,27469114,38,None
FLORIDA,20271272,29,None
NEW YORK,19795791,29,None
ILLINOIS,12859995,20,None


In [16]:
# Updating the Ratio columns with the values

%%sql
UPDATE Election
SET Ratio = Population/GrandElectors;

SELECT * FROM Election LIMIT 5;

 * sqlite://
51 rows affected.
Done.


State,Population,GrandElectors,Ratio
CALIFORNIA,39144818,55,711723.0
TEXAS,27469114,38,722871.0
FLORIDA,20271272,29,699009.0
NEW YORK,19795791,29,682613.0
ILLINOIS,12859995,20,642999.0


In [49]:
# Ordering the states by decreasing ratio of Grand Electors per capita and creating a new table for it. 
# This will make our priority list.

%%sql 

DROP TABLE if EXISTS Election_Sorted;

CREATE TABLE Election_Sorted AS SELECT State, Population, GrandElectors, Ratio 
FROM (SELECT * FROM Election
ORDER BY Ratio DESC);


 * sqlite://
Done.
Done.


[]

In [51]:
# Computing the running total of Grand Electors in that sorted list.
%%sql

SELECT SUM(GrandElectors) FROM Election_Sorted

 * sqlite://
Done.


SUM(GrandElectors)
538


In [52]:
# Independently, computing the half of the total of Grand Electors overall (in the whole country):
# This is the threshold we need to reach for winning the presidential election.

%%sql 
SELECT (SUM(GrandElectors)/2) FROM Election_Sorted

 * sqlite://
Done.


(SUM(GrandElectors)/2)
269


In [130]:
# Filtering and creating a table of our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). 

%%sql

DROP TABLE if EXISTS Election_State_Sorted;

CREATE TABLE Election_State_Sorted AS SELECT State, GrandElectors, Ratio
FROM Election_Sorted
GROUP BY State, GrandElectors
ORDER BY Ratio DESC, State DESC;

 * sqlite://
Done.
Done.


[]

In [131]:
# Checking the table

%%sql 

SELECT * FROM Election_State_Sorted;

 * sqlite://
Done.


State,GrandElectors,Ratio
TEXAS,38,722871.0
CALIFORNIA,55,711723.0
FLORIDA,29,699009.0
NEW YORK,29,682613.0
NORTH CAROLINA,15,669520.0
OHIO,18,645190.0
VIRGINIA,13,644845.0
ILLINOIS,20,642999.0
PENNSYLVANIA,20,640125.0
NEW JERSEY,14,639858.0


In [135]:
# From the above information in that sorted order, we only 12 states to surpass 269 threshold and win the election

In [137]:
# Selecting the top 12 states

%%sql

SELECT State, GrandElectors FROM Election_State_Sorted
ORDER BY Ratio DESC LIMIT 12;

 * sqlite://
Done.


State,GrandElectors
TEXAS,38
CALIFORNIA,55
FLORIDA,29
NEW YORK,29
NORTH CAROLINA,15
OHIO,18
VIRGINIA,13
ILLINOIS,20
PENNSYLVANIA,20
NEW JERSEY,14


## **Conclusion**

**Though 12 is quite a large number but with more information about the states we can narrow our result to less than that. This will enable us to determine the actual swing states. But based on population ratio to the Grand electors, these are thes best states to campaign.**